# Calculation of the Partial Trace

## Background
The partial trace describes the state of a multi-part quantum system after one or more of the parts are discarded. The density matrix of the remaining system is called reduced density matrix and it is calculated with the partial trace formula. For instance, the Bell state $|\Phi^+\rangle=\frac{1}{\sqrt{2}}\left(|00\rangle+|11\rangle\right)$ of two qubits leads to the fully mixed state $\frac{1}{2}\left(|0\rangle \langle 0| + |1\rangle \langle 1|\right)$ for one of the qubits if the other qubit is discarded. Within geqo, discarding one ore more qubits can be achieved with the ```DropQubits``` operation. The simulator ```ensembleSimulatorSymPy``` supports the calculation of partial traces.

## Example without entanglement
The following two examples shows the reduced density matrix of two qubits when there is no entanglement.

In the first example, a Hadamard transform is applied to one of the qubits and the qubit gets immediately discarded after this operation.

In [1]:
from geqo import Sequence
from geqo.gates import Hadamard
from geqo.operations import DropQubits
from geqo.simulators.sympy import ensembleSimulatorSymPy

seq = Sequence([0, 1], [], [(Hadamard(), [0], []), (DropQubits(1), [0], [])])
sim = ensembleSimulatorSymPy(2, 0)
sim.apply(seq, [0, 1])

The density matrix of the system can be accessed with the member variable ```ensemble``` of the backend ```ensembleSimulatorSymPy```. It is a dictionary that contains classical bits as keys. These classical bits are empty, if no measurement was performed so far in the circuit. If there were measurements, then the bit settings correspond to the measurement results. For each bit setting, the probability of this result and the corresponding density matrix can be accessed.

In [2]:
res = sim.ensemble
for r in res:
    prob = res[r][0]
    rho = res[r][1]
    print("probability =", prob)
    display(rho)

probability = 1


Matrix([
[1, 0],
[0, 0]])

The result shows that after discarding the qubit with the Hadamard gate only the initialized qubit remains.

In the following example, the other qubit is discarded. 

In [3]:
seq = Sequence([0, 1], [], [(Hadamard(), [0], []), (DropQubits(1), [1], [])])
sim = ensembleSimulatorSymPy(2, 0)
sim.apply(seq, [0, 1])

res = sim.ensemble
for r in res:
    prob = res[r][0]
    rho = res[r][1]
    print("probability =", prob)
    display(rho)

probability = 1


Matrix([
[1/2, 1/2],
[1/2, 1/2]])

As expected, the density matrix of the qubit with the Hadamard gate is returned. 

In the following example, a third qubit is in the circuit and one qubit is discarded.

In [4]:
seq = Sequence([0, 1, 2], [], [(Hadamard(), [0], []), (DropQubits(1), [1], [])])
sim = ensembleSimulatorSymPy(3, 0)
sim.apply(seq, [0, 1, 2])

res = sim.ensemble
for r in res:
    prob = res[r][0]
    rho = res[r][1]
    print("probability =", prob)
    display(rho)

probability = 1


Matrix([
[1/2, 0, 1/2, 0],
[  0, 0,   0, 0],
[1/2, 0, 1/2, 0],
[  0, 0,   0, 0]])

The result shows that two qubits remain and that one qubit without Hadamard gate remains.

In the following example, there are three qubits and the Hadamard transform is applied to one. After this, the two other qubits are discarded.

In [5]:
seq = Sequence([0, 1, 2], [], [(Hadamard(), [0], []), (DropQubits(2), [1, 2], [])])
sim = ensembleSimulatorSymPy(3, 0)
sim.apply(seq, [0, 1, 2])

res = sim.ensemble
for r in res:
    prob = res[r][0]
    rho = res[r][1]
    print("probability =", prob)
    display(rho)

probability = 1


Matrix([
[1/2, 1/2],
[1/2, 1/2]])

The result shows that just the qubit with the Hadamard transform remains.

## Example with entanglement
In the following example, a Hadamard transform is applied to a qubit leading to a superposition of $|0\rangle$ and $|1\rangle$. After this, the qubit is used as a control qubit of a CNOT operation with two qubits in a row. This leads to three entangled qubits and in the following circuit one of the qubits is discarded.

In [6]:
from geqo.gates import CNOT

seq = Sequence(
    [0, 1, 2],
    [],
    [
        (Hadamard(), [0], []),
        (CNOT(), [0, 1], []),
        (CNOT(), [0, 2], []),
        (DropQubits(1), [0], []),
    ],
)

sim = ensembleSimulatorSymPy(3, 0)
sim.apply(seq, [0, 1, 2])

res = sim.ensemble
for r in res:
    prob = res[r][0]
    rho = res[r][1]
    print("probability =", prob)
    display(rho)

probability = 1


Matrix([
[1/2, 0, 0,   0],
[  0, 0, 0,   0],
[  0, 0, 0,   0],
[  0, 0, 0, 1/2]])

The result is a mixed state of $|00\rangle$ and $|11\rangle$ as expected, because discarding one part of three entangled qubits leads to a state of the system, which is not pure, but the two remaining qubits are correlated.

Finally, the following example is the same, but two qubits are discarded instead of one.

In [7]:
seq = Sequence(
    [0, 1, 2],
    [],
    [
        (Hadamard(), [0], []),
        (CNOT(), [0, 1], []),
        (CNOT(), [0, 2], []),
        (DropQubits(2), [0, 1], []),
    ],
)

sim = ensembleSimulatorSymPy(3, 0)
sim.apply(seq, [0, 1, 2])

res = sim.ensemble
for r in res:
    prob = res[r][0]
    rho = res[r][1]
    print("probability =", prob)
    display(rho)

probability = 1


Matrix([
[1/2,   0],
[  0, 1/2]])

The result is a qubit with a fully mixed state.